In [1]:
import os
import pandas as pd
from PIL import Image

import wandb
import math

from ipywidgets import IntProgress
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import f1_score

import sys
from glob import glob
import numpy as np
import pandas as pd
import cv2
from PIL import Image
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
from time import time

import torch
import torch.utils.data as data

import matplotlib.pyplot as plt
import seaborn as sns
from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import albumentations.pytorch

import matplotlib.pyplot as plt
import seaborn as sns
import random

import timm
import sklearn.metrics

In [2]:
device = torch.device("cuda:0")
batch_size = 16
class_n = 2
learning_rate = 1e-6
epochs = 20
save_path = 'models/'
class_n

2

In [3]:
os.environ['WANDB_NOTEBOOK_NAME'] = 'code.ipynb'
wandb.init(project='NPDH', name='efficientnet-b2_kfold1_augmentation',
    config={
    "batch size": batch_size,
    "epochs" : epochs,
    "learning rate": learning_rate,
    # "backborn" : 'se_resnext101_32x4d', 
    # "architecture" : 'Deeplabv3+', 
})

wandb: Currently logged in as: boostcampcv08 (use `wandb login --relogin` to force relogin)


In [4]:
def evaluation(ground_truth, pred_output, labels = [1,0]):
    mtris = sklearn.metrics.confusion_matrix(ground_truth,pred_output, labels =[1,0])
    TP, FN = mtris[0]
    FP, TN = mtris[1]
    ttl = TP+FN+FP+TN
    accuracy = (TP+TN)/ttl
    specificity = TN/(TN+FP)
    sensitivity = TP(TP+FN)
    precision = TP(TP+FP)
    negative_predicable_value = TN/(TN+FN)
    F1score = 2*precision*sensitivity/(precision+sensitivity)

    return accuracy,specificity,sensitivity,precision,negative_predicable_value,F1score

In [5]:
# Set random seed
import random
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)  # type: ignore
torch.backends.cudnn.deterministic = True  # type: ignore
torch.backends.cudnn.benchmark = True  # type: ignore

In [6]:
base_path = os.path.join(os.getcwd(),'a-trac-gi/final')
# colon_negative = 'colon_negative'
# colon_negative_test = 'colon_negative_test'
# colon_positive = 'colon_positive'
gi_test = 'gi_test_set'

In [7]:
# train_negative_list = os.listdir(os.path.join(base_path,colon_negative))
# test_negative_list = os.listdir(os.path.join(base_path,colon_negative_test))
# train_positive_list = os.listdir(os.path.join(base_path,colon_positive))
gi_test_list = os.listdir(os.path.join(base_path,gi_test))

In [8]:
gi_test_list = sorted(gi_test_list)

In [9]:
# train_nagative = [(img_name, os.path.join(base_path,colon_negative,img_name),0) for img_name in train_negative_list]
# train_positive = [(img_name, os.path.join(base_path,colon_positive,img_name),1) for img_name in train_positive_list]
# train_pd = pd.DataFrame(train_nagative+train_positive, columns = ['image_id','image_path','labels'])
# train_pd.to_csv('train_pd.csv')
# train_pd = train_pd.sample(frac=1).reset_index(drop=True)

In [10]:
test_positive = [(img_name, os.path.join(base_path,'gi_test_set',img_name),1) for img_name in gi_test_list]
test_dataset = pd.DataFrame(test_positive, columns = ['image_id','image_path','labels'])
test_dataset.to_csv('test_pd.csv')
# test_pd = test_pd.sample(frac=1).reset_index(drop=True)

In [11]:
train_total = pd.read_csv("train_1_5.csv")
test = pd.read_csv("test_1_5.csv")

In [12]:
 = pd.read_csv("test_1_5.csv")
test_dataset['image_path'][0]

'/home/ec2-user/dataset/a-trac-gi/final/gi_test_set/Image_0001.jpg'

## 데이터셋 구성

In [13]:
class CustomDataset(Dataset):
    def __init__(self, files, labels=None, mode='train', transform = None):
        self.mode = mode
        self.files = files
        self.transform = transform
        if mode == 'train':
            self.labels = labels
            
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, i):
        if self.mode == 'train':
            img = cv2.imread(self.files[i])
            img = cv2.resize(img, dsize=(512, 512), interpolation=cv2.INTER_AREA)
            img = img.astype(np.float32)/255
            # img = np.transpose(img, (2,0,1))
            if self.transform is not None:
                transformed = self.transform(image=img)
                img = transformed["image"]
                
            return {
                'img' : img,
                'label' : torch.tensor(self.labels[i], dtype=torch.long)
            }
        else:
            img = cv2.imread(self.files[i])
            img = cv2.resize(img, dsize=(512, 512), interpolation=cv2.INTER_AREA)
            img = img.astype(np.float32)/255
            if self.transform is not None:
                transformed = self.transform(image=img)
                img = transformed["image"]
            # img = np.transpose(img, (2,0,1))
            return {
                'img' : img
            }

In [14]:
train = train_total
val = test

In [15]:
# train,val

In [16]:
train_transform = A.Compose([
                    A.Cutout(num_holes=8, max_h_size=8, max_w_size=8, fill_value=1, always_apply=False, p=0.05),
                    A.Rotate(always_apply=False, p=1.0, limit=(-90, 90), interpolation=0, border_mode=0, value=(255, 255, 255), mask_value=None),
                    A.RandomResizedCrop(always_apply=False, p=0.1, height=512, width=512, scale=(0.5, 1.0), ratio=(0.75, 1.3333333730697632), interpolation=0),
                    A.HorizontalFlip(p=0.1),
                    A.VerticalFlip(p=0.1),
                    A.HueSaturationValue(always_apply=False, p=0.2, hue_shift_limit=(-20, 20), sat_shift_limit=(-30, 30), val_shift_limit=(-20, 20)),
                    A.ShiftScaleRotate(shift_limit=0.2, scale_limit=0.2, rotate_limit=10, border_mode=4, p=0.1),
                    A.ElasticTransform(p=0.2),
                    ToTensorV2()
                          ])
val_transform = A.Compose([
                          ToTensorV2()
                          ])


/home/ec2-user/anaconda3/envs/yh/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:689: FutureWarning: This class has been deprecated. Please use CoarseDropout
  warnings.warn(


In [17]:
train_dataset = CustomDataset(train['image_path'], train['labels'].values, transform = train_transform)
val_dataset = CustomDataset(val['image_path'], val['labels'].values, transform = val_transform)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, num_workers=0, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, num_workers=0, shuffle=False)

test_dataset = CustomDataset(test['image_path'], labels=None, mode='test')
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, num_workers=0, shuffle=False)

In [19]:
# from pprint import pprint
# model_names = timm.list_models(pretrained=True)
# pprint(model_names)

In [20]:
# from efficientnet_pytorch import EfficientNet

# m = EfficientNet.from_pretrained('efficientnet-b2',num_classes=2)
# print("네트워크 필요 입력 채널 개수", m._conv_stem.weight.shape[1])
# print("네트워크 출력 채널 개수 (예측 class type 개수)", m._fc.weight.shape[0])
# m.eval()

In [21]:

# model = timm.create_model('coat_tiny', pretrained=True)
# model.head.out_features = 2
# model.eval()


from efficientnet_pytorch import EfficientNet
model = EfficientNet.from_pretrained('efficientnet-b0',num_classes=2)

print("네트워크 필요 입력 채널 개수", model._conv_stem.weight.shape[1])
print("네트워크 출력 채널 개수 (예측 class type 개수)", model._fc.weight.shape[0])

# # print("네트워크 필요 입력 채널 개수", model._conv_stem.weight.shape[1])
# print("네트워크 출력 채널 개수 (예측 class type 개수)", model._fc.weight.shape[0])

Loaded pretrained weights for efficientnet-b0
네트워크 필요 입력 채널 개수 3
네트워크 출력 채널 개수 (예측 class type 개수) 2


In [22]:
model = model.to(device)

In [23]:
model.to(device)
# learning_rate = 0.0001 
# NUM_EPOCH = 5
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,'min')

In [24]:
def evaluation(ground_truth, pred_output, labels = [1,0]):
    mtris = sklearn.metrics.confusion_matrix(ground_truth,pred_output, labels=labels)
    TP, FN = mtris[0]
    FP, TN = mtris[1]
    ttl = TP+FN+FP+TN
    accuracy = (TP+TN)/ttl
    specificity = TN/(TN+FP)
    sensitivity = TP/(TP+FN)
    precision = TP/(TP+FP)
    negative_predicable_value = TN/(TN+FN)
    F1score = 2*precision*sensitivity/(precision+sensitivity)

    return accuracy,specificity,sensitivity,precision,negative_predicable_value,F1score

def train_step(batch_item, epoch, batch, training):
    img = batch_item['img'].to(device)
    label = batch_item['label'].to(device)
    ground_truth = []
    pred_output = []
    if training is True:
        model.train()
        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            output = model(img)
            loss = criterion(output, label)
            pred_output+=output.argmax(axis =1).cpu().tolist()
            ground_truth+=label.cpu().tolist()
        loss.backward()
        optimizer.step()
        return loss,ground_truth,pred_output
    else:
        model.eval()
        with torch.no_grad():
            output = model(img)
            loss = criterion(output, label)
            pred_output+=output.argmax(axis =1).cpu().tolist()
            ground_truth+=label.cpu().tolist()
        return loss,ground_truth,pred_output



In [25]:
save_path = '/home/ec2-user/dataset/kfold/efficientnet-kfold1_transform.pth'

In [26]:
save_path = '/home/ec2-user/dataset/kfold/efficientnet-kfold1_transform.pth'
model = torch.load(save_path)
model.to(device)

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=

In [27]:
loss_plot, val_loss_plot = [], []

for epoch in range(epochs):
    total_loss, total_val_loss = 0, 0
    
    tqdm_dataset = tqdm(enumerate(train_dataloader))
    training = True
    ground_truth = []
    pred_output = []   
    for batch, batch_item in tqdm_dataset:
        # print(batch_item.keys())
        batch_loss,gt,po = train_step(batch_item, epoch, batch, training)
        total_loss += batch_loss
        ground_truth+=gt
        pred_output+=po
        tqdm_dataset.set_postfix({
            'Epoch': epoch + 1,
            'Loss': '{:06f}'.format(batch_loss.item()),
            'Total Loss' : '{:06f}'.format(total_loss/(batch+1))
        })
    loss_plot.append(total_loss/(batch+1))
    acc,spec,sens,prec,npv,f1 = evaluation(ground_truth,pred_output)

    wandb.log({
            'train Total Loss' : '{:06f}'.format(total_loss/(batch+1)),
            "train Accuracy" : acc, 
            "train Specificity" : spec,
            "train Sensitivity" : sens,
            "train Precision" : prec,
            "train Negative_Predicable_Value" : npv,
            "train F1score" : f1,
            "train total_mean" : (acc+spec+sens+prec+npv+f1)/6
        })
    tqdm_dataset = tqdm(enumerate(val_dataloader))
    training = False
    ground_truth = []
    pred_output = []   
    for batch, batch_item in tqdm_dataset:
        batch_loss,gt,po = train_step(batch_item, epoch, batch, training)
        total_val_loss += batch_loss
        ground_truth+=gt
        pred_output+=po       
        tqdm_dataset.set_postfix({
            'Epoch': epoch + 1,
            'Val Loss': '{:06f}'.format(batch_loss.item()),
            'Total Val Loss' : '{:06f}'.format(total_val_loss/(batch+1))
        })
    acc,spec,sens,prec,npv,f1 = evaluation(ground_truth,pred_output)
    wandb.log({
            "val Total Loss": '{:06f}'.format(total_val_loss/(batch+1)),
            "val Accuracy" : acc, 
            "val Specificity" : spec,
            "val Sensitivity" : sens,
            "val Precision" : prec,
            "val Negative_Predicable_Value" : npv,
            "val F1score" : f1,
            "val total_mean" : (acc+spec+sens+prec+npv+f1)/6
    })
    val_loss_plot.append(total_val_loss/(batch+1))
    
    if min(val_loss_plot) == val_loss_plot[-1]:
        torch.save(model, save_path)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [23]:
len(pred_output),len(ground_truth)

NameError: name 'pred_output' is not defined

In [44]:
from efficientnet_pytorch import EfficientNet
model = EfficientNet.from_pretrained('efficientnet-b0',num_classes=2)


(1.0, 1.0, 1.0, 1.0, 1.0, 1.0)

In [65]:
save_path = '/home/ec2-user/dataset/kfold/efficientnet-kfold1_transform.pth'
model = torch.load(save_path)
# model.eval()

In [71]:
test_data = pd.read_csv('test_pd.csv')
test_transform = A.Compose([
                          ToTensorV2()
                          ])

In [73]:
test_dataset = CustomDataset(test_data['image_path'], labels=None, mode='test',transform = test_transform)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=1, num_workers=0, shuffle=False)
# val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, num_workers=0, shuffle=False)

# test_inference
tqdm_test_dataset = tqdm(enumerate(test_dataloader))
pred_output = []
ground_truth = []
for batch, batch_item in tqdm_test_dataset:
    img = batch_item['img'].to(device)
    # label = batch_item['label'].to(device)
    with torch.no_grad():
        output = model(img)
        pred_output+=output.argmax(axis =1).cpu().tolist()
        # ground_truth+=label.cpu().tolist()

0it [00:00, ?it/s]

In [74]:
import csv
with open('kfold/efficientnet-kfold1_transform.csv', 'w') as f:
    write = csv.writer(f)
    write.writerow(pred_output)

AttributeError: 'list' object has no attribute 'to_csv'